# Exercise 5 - Automatically find Exchange links for DWM Query

In this demo we will see how to combine DWM and Iknaio to automatically find connections to exchanges given a bunch of crypto addresses mentioned some darkweb context

## Preparations

First, we install the graphsense-python package and define an API-key. An API-key for the [GraphSense](https://graphsense.github.io/) instance hosted by [Iknaio](https://www.ikna.io/) can be requested by sending an email to [contact@iknaio.com](contact@iknaio.com).

In [65]:
!pip install graphsense-python seaborn

import graphsense
from graphsense.api import bulk_api, general_api

import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [66]:
# load config.json
with open('../config.json') as f:
    config = json.load(f)


configuration = graphsense.Configuration(
    host = "https://api.ikna.io/",
    api_key = {
        'api_key': config['graphsense']['api_key']
    }
)

GraphSense supports Bitcoin-like UTXO and Ethereum-like Account-Model ledgers. Iknaio currently hosts BTC, LTC, BCH, ZEC, and ETH.

We are investigating Bitcoin transactions, therefore we set the default currency to Bitcoin **BTC**.

In [67]:
CURRENCY = 'btc'

We can test whether or client works, by checking what data the GraphSense endpoint provides

In [68]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = general_api.GeneralApi(api_client)
    api_response = api_instance.get_statistics()
    display(api_response)

{'currencies': [{'name': 'btc',
                 'no_address_relations': 10702744647,
                 'no_addresses': 1362721808,
                 'no_blocks': 879056,
                 'no_entities': 619018653,
                 'no_labels': 28866,
                 'no_tagged_addresses': 313255850,
                 'no_txs': 1143075248,
                 'timestamp': 1736754212},
                {'name': 'bch',
                 'no_address_relations': 2815733259,
                 'no_addresses': 357464141,
                 'no_blocks': 880818,
                 'no_entities': 166550051,
                 'no_labels': 436,
                 'no_tagged_addresses': 15707850,
                 'no_txs': 405436925,
                 'timestamp': 1736738682},
                {'name': 'ltc',
                 'no_address_relations': 2218283863,
                 'no_addresses': 321713100,
                 'no_blocks': 2826418,
                 'no_entities': 164551776,
                 'no_labels': 5

# 1. Load Starting Addresses from DWM


In [69]:
addresses = pd.read_csv(
    'https://raw.githubusercontent.com/iknaio/iknaio-api-tutorial/main/data/sextortion_addresses.csv',
    header=None,
    names=["address"]
)
addresses

,address
0,1JwRp2J8bQcoG8XTUbxQZaEj9QB4RB6zEa
1,1EZS92K4xJbymDLwG4F7PNF5idPE62e9XY
2,16B4HuSAJ4WRdCq7dzA5b4ASh6QQ7ytZWB
3,1EdX5vtBiHGmkqbJc7VRSuVMx9Kpgh53Tp
4,3Ch7RPfwkJ3wHhiBfA4CNc8SagGdjbZwVs
...,...
240,1BC1pvPUQF9QHg73ha4AEAhaoEvg6HmTbS
241,13QKq8RsvbJnLRbi5ZcVX1ziYW83tqvp1q
242,1DiEqE5R1Ktu7QCLUuJN31PNtpoBU41x2E
243,1NWybUp8ZJXKyDg2DR5MaePspforMPYbM3


# Q1. How many of the addresses are used?

Instead of querying each address individually, we just pass the dataframe of known sextortion payment addresses.

In [70]:
with graphsense.ApiClient(configuration) as clnt:
    blkapi = bulk_api.BulkApi(clnt)

    # documentation about available bulk operations can be found
    # here https://api.ikna.io/#/bulk/bulk_csv
    rcsv = blkapi.bulk_csv(
                CURRENCY,
                operation="get_address",
                body={
                    'address': addresses['address'].to_list()
                },
                num_pages=1,
                _preload_content=False
              )
    respAddrDF = pd.read_csv(rcsv)
respAddrDF[["address", "balance_eur", "total_received_eur", "total_spent_eur", "in_degree", "out_degree", "no_incoming_txs", "no_outgoing_txs", "first_tx_height", "last_tx_height"]]

,address,balance_eur,total_received_eur,total_spent_eur,in_degree,out_degree,no_incoming_txs,no_outgoing_txs,first_tx_height,last_tx_height
0,12Mr26cq6CnMFyG93ig6tNNR1wRDTpr1Jk,0.0,905.15,533.58,3,1,2,1,547996,554513
1,17b1VPxMsea5u6eLQ8d5w5RkaG7hmtYvBP,0.0,790.05,463.54,1,1,1,1,548324,554513
2,1KuUfck3gAcyABNG12MpJTZyraemMn45oF,0.0,801.92,466.40,6,1,1,1,549944,554513
3,19kjHnoEdtytyGHuNxEw83fkFtW5ERoQVh,0.0,811.86,472.05,2,1,2,1,549561,554513
4,1D2NtiTeH381PbbdpemDKaYx3LScd9kfQ4,0.0,491.59,478.71,1,1,1,1,551749,554513
...,...,...,...,...,...,...,...,...,...,...
240,1DzM9y4fRgWqpZZCsvf5Rx4HupbE5Q5r4y,0.0,8524.76,8667.58,22,4,18,2,543026,545197
241,1FgfdebSqbXRciP2DXKJyqPSffX3Sx57RF,0.0,17100.15,16306.23,65,5,29,4,551672,566798
242,1HWnuWz77oXgbZLVBfB9fQAAgLCJJS7m7T,0.0,695.29,696.35,2,2,1,1,544376,545195
243,1ZWFbUTUEQw7VMCzWu1SzfPm9HaiqP6rX,0.0,1534.20,1161.10,3,2,3,2,548067,560647


In [71]:
print(f"{len(respAddrDF)} addresses received {sum(respAddrDF['total_received_usd']):.2f} USD")

245 addresses received 886359.19 USD


# Q2: Are there direct links to exchanges?

In [75]:
import requests
import time
from tqdm import tqdm

seconds = 40
address = "15sJ8z2VXR6T7spiDj2W7UJbjJhqWpy33W" # todo
max_depth = 30
max_breadth = 100


header = {
    "cookie": f"remember_prod={config['graphsense']['session']}"
}

def get_QL_results(address: str) -> dict:

    def get_task_id(address: str) -> str:
        rq = f"https://api.ikna.io/quicklock/follow_flows_to_exchange/{CURRENCY}?perpetrator_address={address}&max_search_depth={max_depth}&max_search_breadth={max_breadth}&search_time_seconds={seconds}"

        response = requests.get(rq, headers=header)
        response.json()
        return response.json()['task_id']

    def get_data(task_id):

        response_got = False
        while not response_got:
            req = f"https://api.ikna.io/quicklock/get_task_state/{task_id}?include_path_details=false"
            response = requests.get(req, headers=header)
            response_json = response.json()
            if response_json['state'] in ["done", "timeout"]:
                response_got = True
            else:
                time.sleep(2)

        rq_get_result = f"https://api.ikna.io/quicklock/get_task_state/{task_id}?include_path_details=true"

        response = requests.get(rq_get_result, headers=header)
        result = response.json()
        results = result['results']
        data = {
            "address": address,
            "pct_traced_to_exchange": results['pct_traced_to_exchange'],
            "nr_pathes_found": results['nr_pathes_found'],
            "paths": results['paths']
        }
        return data

    task_id = get_task_id(address)
    return get_data(task_id)

address_list = addresses['address'].to_list()[:10]

results_list = []
for address in tqdm(address_list, desc="Processing addresses"):
    results_list.append(get_QL_results(address))

df_ql = pd.DataFrame(results_list)
df_ql

Processing addresses: 100%|██████████| 10/10 [00:46<00:00,  4.62s/it]


,address,pct_traced_to_exchange,nr_pathes_found,paths
0,1JwRp2J8bQcoG8XTUbxQZaEj9QB4RB6zEa,3.109556,9,[{'nodes': [{'input_address': '3CneU2wfS8Lf27u...
1,1EZS92K4xJbymDLwG4F7PNF5idPE62e9XY,0.000000,0,None
2,16B4HuSAJ4WRdCq7dzA5b4ASh6QQ7ytZWB,0.000000,0,None
3,1EdX5vtBiHGmkqbJc7VRSuVMx9Kpgh53Tp,45.495085,19,[{'nodes': [{'input_address': 'bc1qrs5tk94a6ct...
4,3Ch7RPfwkJ3wHhiBfA4CNc8SagGdjbZwVs,0.000000,0,None
5,19GqTJDhu7A1qg7rnK3KS7tmCkCTMTz6xD,0.013151,2,[{'nodes': [{'input_address': '3BgUG3nChgYR4hE...
6,1NMRCQMfhfVyAyuEubdfneE2H458Njog3v,0.000000,0,None
7,16XhmM7nPvR15eFdmVJs4QWcWpnYVS6FTv,0.000000,0,None
8,1EyXwmxKd74HeyqbZbmXJsNxmfpiPeAF3F,0.000000,0,None
9,1NPy1TBRyk6vMeGG3e5GaJWxYa9HbsNtDm,21.851681,1,[{'nodes': [{'input_address': 'bc1qt7v2jl0l2ek...


In [91]:
# get the first paths that is not None
paths = next(filter(None, df_ql['paths']))
paths

# get the first path of the paths
path = paths[0]["nodes"]
addresses = [node["output_address"] for node in path]
transactions = [node["tx_hash"] for node in path]

# assemble the trace string
transaction_prefix = "T_"
perpetrator_prefix = "PA_"
neutral_prefix = "HA_"
trace_str = f"{perpetrator_prefix}{addresses[0]}"
n_txs = len(transactions)
for i in range(1, n_txs):
    trace_str += f",T_{transactions[i]},{neutral_prefix}{addresses[i]}"


url = f"https://app.ikna.io/pathfinder/btc/path/{trace_str}"
url

'https://app.ikna.io/pathfinder/btc/path/PA_1JwRp2J8bQcoG8XTUbxQZaEj9QB4RB6zEa,T_db7adde440d9f1ee69938cffeededc5b6917e79fb6263dc632546ae33a4a78d9,HA_1JwRp2J8bQcoG8XTUbxQZaEj9QB4RB6zEa,T_22da5847109363a6193b36558263a509fb48c2c0e3438dfdbb6c58141a1a712e,HA_38YEQmUhQWb37JzhNJSmXtXQhKy7adAcNu,T_24c3471a977673be1246112aabed3547204ca3bc20cbec747618c638dce34372,HA_bc1ql4909z0h0jhtr4ntuz6l2fndc7pdf8rflssz8s,T_e0fcded3fa053197403825a55fc07d2099eafbf4ea15f9614a38cbfe318a220c,HA_3L4KtC4QDY4hANF8J7zc2KqrnBuJdgyvkc,T_9856b203625e7b09f8d16d198315403542dcb4d68bd20d4784c2ac0369ad175e,HA_3Guu5tUGZBCQdwX6zLyDhgDrHX9GZ14DKX,T_b8ed67db6f7396f4b1183d4bb44507149acccf79c38cb8bc1138caeaff2118d1,HA_3ANXMo5pCSMCKykATA4ogH3eY2qVsyu4qM'

# Q3: Can i link more addresses to our seed addresses?

We now fetch summary statistics for each entity.

In [73]:
with graphsense.ApiClient(configuration) as clnt:
  blkapi = bulk_api.BulkApi(api_client)
  rcsv = blkapi.bulk_csv(
                                 CURRENCY,
                                 operation = "get_entity",
                                 body={
                                     'entity': respAddrDF['entity'].drop_duplicates().to_list()
                                     },
                                 num_pages=1,
                                 _preload_content=False
                                 )
  respEntityDF = pd.read_csv(rcsv)

respEntityDF[
    ["best_address_tag_label",
     "root_address",
     "no_addresses",
     "balance_eur",
     "total_received_eur",
     "total_spent_eur",
     "in_degree",
     "out_degree",
     "no_incoming_txs",
     "no_outgoing_txs",
     "first_tx_height",
     "last_tx_height"]
     ]

,best_address_tag_label,root_address,no_addresses,balance_eur,total_received_eur,total_spent_eur,in_degree,out_degree,no_incoming_txs,no_outgoing_txs,first_tx_height,last_tx_height
0,Sextortion Spam,17YQspbjPuCR65TUtEDqsc37qVZKR7zuBJ,2,0.00,1.28,1.28,1,2,1,1,557906,557906
1,Sextortion Spam,12VA3fnjCkJBYwgxffD138xXqFNYTWwV5w,2,0.00,1.31,1.32,2,1,2,1,557906,557906
2,Sextortion Spam,12L7czMjP1P9Sd35of8jRsbgVcLmy22LQ2,4,0.00,1588.48,1727.12,4,1,4,1,558400,564600
3,Sextortion Spam,1BzkoGfrLtL59ZGjhKfvBwy47DEb6oba5f,22,3.04,160123.50,158363.14,255,14,299,13,548022,566798
4,Sextortion Spam,1DwFLh8ChA1b1hhyW4DdoE9aQXEpUTZesB,1,0.00,352.27,352.27,1,2,1,1,557712,557713
...,...,...,...,...,...,...,...,...,...,...,...,...
91,Sextortion Spam,1EFBBqVxZ4H71TJXJDD7KNPpWMs35kTdVw,6,33.10,10214.94,9763.91,17,2,21,2,552624,792228
92,Sextortion Spam,1EaqpQL5AoCb6iuPhiRx1urzDV1MUo9AWd,9,0.00,3964.38,3958.52,17,14,17,9,541800,553302
93,Sextortion Spam,1En9DGURuyS4oojZEAFJwFLBtmsBEc8r8x,31,0.00,48067.02,63398.03,30,5,34,5,540034,576800
94,Sextortion Spam,1A7nmeXMjFd8unyg9hopifwBRpWu5MpqQw,2,0.00,1404.87,1341.22,3,1,7,1,553282,553642
